LLAVA case study

In [2]:
# Load model directly
from transformers import AutoProcessor, AutoModelForImageTextToText, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
import torch
import pandas as pd
from PIL import Image
from torch.amp import autocast

device = "cpu"
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
model = AutoModelForImageTextToText.from_pretrained("llava-hf/llava-1.5-7b-hf").to(device)

c:\Users\Seth\miniconda3\envs\nlpproj\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 
Loading checkpoint shards: 100%|██████████| 3/3 [01:24<00:00, 28.18s/it]


load dataset

In [3]:
train_dataset = pd.read_csv("generated_img_dataset/train.csv")
valid_dataset = pd.read_csv("generated_img_dataset/val.csv")
test_dataset = pd.read_csv("generated_img_dataset/test.csv")

In [6]:
test_dataset

,EM,EN,unicode,label,strategy,image
0,👔📈,big business,U+1F454 U+1F4C8,1,1,./generated_img_dataset/test_google/0.png
1,🏢🤑🤑,big business,U+1F3E2 U+1F911 U+1F911,1,1,./generated_img_dataset/test_google/1.png
2,👨‍💻🤝,big business,U+1F468 U+200D U+1F4BB U+1F91D,1,1,./generated_img_dataset/test_google/2.png
3,🏢🧑‍🤝‍🧑🧑‍🤝‍🧑🧑‍🤝‍🧑,big business,U+1F3E2 U+1F9D1 U+200D U+1F91D U+200D U+1F9D1 ...,1,1,./generated_img_dataset/test_google/3.png
4,👩‍💻🤑,big business,U+1F469 U+200D U+1F4BB U+1F911,1,1,./generated_img_dataset/test_google/4.png
...,...,...,...,...,...,...
488,👍👣,effective entrance,U+1F44D U+1F463,0,6,./generated_img_dataset/test_google/513.png
489,👏🪜,effective entrance,U+1F44F U+1FA9C,0,6,./generated_img_dataset/test_google/514.png
490,😤🗣️💬,effective entrance,U+1F624 U+1F5E3 U+FE0F U+1F4AC,0,6,./generated_img_dataset/test_google/515.png
491,💨🤬,effective entrance,U+1F4A8 U+1F92C,0,6,./generated_img_dataset/test_google/516.png


In [ ]:
import torch
from PIL import Image
from tqdm.auto import tqdm

# --- Configuration ---
BATCH_SIZE = 8 # <<< START SMALL (e.g., 2 or 4) and increase if possible

# Helper function to yield batches from the DataFrame
def generate_batches(df, batch_size):
    for i in range(0, len(df), batch_size):
        yield df.iloc[i:min(i + batch_size, len(df))]

def batch_zero_shot_predict(batch_samples):
    """
    Given a batch of samples (a slice of your DataFrame),
    construct conversations, generate prompts, process images and text
    through the model to produce zero-shot predictions for the batch.
    Returns a tuple (list_of_predictions, list_of_generated_texts).
    """
    batch_prompts_structured = []
    batch_raw_images = []
    batch_indices_processed = [] # Keep track of which samples were successful

    # 1. Prepare batch data (prompts and images)
    for index, sample in batch_samples.iterrows():
        # Construct prompt message
        prompt_message = f"Does this emoji sequence mean '{sample['EN']}'? Answer yes or no."

        # Build the conversation structure for the processor template
        # Note: We still build one structure per sample before applying the template
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_message},
                    {"type": "image"},
                ],
            },
        ]
        batch_prompts_structured.append(conversation)

        # Load the image
        try:
            raw_image = Image.open(sample['image']).convert("RGB")
            batch_raw_images.append(raw_image)
            batch_indices_processed.append(index) # Store original index
        except Exception as e:
            print(f"Warning: Error loading image {sample['image']} for index {index}: {e}. Skipping this sample.")
            # Append placeholders to keep lists aligned for now, will filter later
            # Or handle differently if your processor/model is sensitive to None
            batch_raw_images.append(None) # Placeholder

    # Filter out samples where image loading failed
    valid_indices = [i for i, img in enumerate(batch_raw_images) if img is not None]
    if not valid_indices:
        print("Warning: No valid images found in this batch. Skipping.")
        # Return empty lists matching the expected output structure
        return [0] * len(batch_samples), ["<IMAGE_LOAD_ERROR>"] * len(batch_samples)

    # Keep only valid images and their corresponding structured prompts
    final_batch_raw_images = [batch_raw_images[i] for i in valid_indices]
    final_batch_prompts_structured = [batch_prompts_structured[i] for i in valid_indices]
    original_indices_for_valid = [batch_samples.index[i] for i in valid_indices] # Get original df index

    # 2. Apply chat template and tokenize prompts
    # We need to apply the template individually then tokenize as a batch
    # because apply_chat_template usually works on a single conversation.
    batch_final_prompts_text = []
    for conv in final_batch_prompts_structured:
         # Use add_generation_prompt=True for the model's turn
        prompt_text = processor.apply_chat_template(conv, add_generation_prompt=True)
        batch_final_prompts_text.append(prompt_text)

    # 3. Process the batch of images and text prompts
    # Use padding=True for batching text inputs
    inputs = processor(
        images=final_batch_raw_images,
        text=batch_final_prompts_text,
        return_tensors="pt",
        padding=True, # Crucial for batching text
        truncation=True # Good practice
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 4. Generate outputs from the model for the batch
    generated_ids = None
    try:
        with torch.no_grad():
            generated_ids = model.generate(**inputs, max_new_tokens=50)
    except Exception as e:
         print(f"Error during model generation for a batch (possible OOM): {e}")
         # Handle error: return default predictions for this batch
         # Create placeholder results aligned with the original batch size
         batch_predictions = [0] * len(batch_samples)
         batch_generated_texts = ["<GENERATION_ERROR>"] * len(batch_samples)
         return batch_predictions, batch_generated_texts

    # Clean up GPU memory if applicable
    # del inputs
    # if torch.cuda.is_available():
    #     torch.cuda.empty_cache()

    # 5. Decode the generated tokens to text for the batch
    # Use batch_decode for efficiency
    batch_decoded_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)

    # 6. Interpret results and map back to original batch structure
    batch_predictions_map = {}
    batch_generated_texts_map = {}
    for i, decoded_text in enumerate(batch_decoded_texts):
        cleaned_text = decoded_text.strip().lower()
        # The LLaVA output might include the prompt, need to isolate the answer
        # Find the last occurrence of the assistant's turn marker if apply_chat_template added one
        # Or simply check the end of the string if the template is simple
        # Example check (adapt based on your model's exact output format):
        answer_part = cleaned_text.split("assistant:")[-1] # separator

        prediction = 1 if "yes" in answer_part else 0
        original_index = original_indices_for_valid[i] # Map back using original df index
        batch_predictions_map[original_index] = prediction
        batch_generated_texts_map[original_index] = cleaned_text # Store full generated text

    # Create final lists in the original batch order, filling in defaults for skipped items
    
    final_predictions = []
    final_gen_texts = []
    for index in batch_samples.index:
        final_predictions.append(batch_predictions_map.get(index, 0)) # Default 0 if skipped/error
        final_gen_texts.append(batch_generated_texts_map.get(index, "<SKIPPED_OR_ERROR>"))

    return final_predictions, final_gen_texts

# --- Main prediction loop using batches ---
predictions = []
generated_texts = []
true_labels = test_dataset["label"].tolist()

# Create the batch generator with a progress bar
batch_generator = generate_batches(test_dataset, BATCH_SIZE)
num_batches = (len(test_dataset) + BATCH_SIZE - 1) // BATCH_SIZE

print(f"Starting prediction with batch size {BATCH_SIZE} on device {device}...")
for batch_df in tqdm(batch_generator, total=num_batches, desc="Processing Batches"):
    batch_preds, batch_gen_texts = batch_zero_shot_predict(batch_df)
    predictions.extend(batch_preds)
    generated_texts.extend(batch_gen_texts)

    # Optional: Print progress for the first item in the batch
    first_idx = batch_df.index[0]
    print(f"Batch starting row {first_idx}: EN = {batch_df.iloc[0]['EN']} -> Prediction: {batch_preds[0]}, Generated: {batch_gen_texts[0][:100]}...") # Print truncated generated text

print("Finished prediction.")

# from sklearn.metrics import accuracy_score, f1_score
# print(f"Accuracy: {accuracy_score(true_labels, predictions)}")
# print(f"F1 Macro: {f1_score(true_labels, predictions, average='macro')}")


Starting prediction with batch size 8 on device cpu...


Processing Batches:   2%|▏         | 1/62 [11:29<11:40:53, 689.40s/it]

Batch starting row 0: EN = big business -> Prediction: 1, Generated: user:  
does this emoji sequence mean 'big business'? answer yes or no. assistant: yes....


Processing Batches:   3%|▎         | 2/62 [19:14<9:17:37, 557.62s/it] 

Batch starting row 8: EN = big expenditure -> Prediction: 1, Generated: user:  
does this emoji sequence mean 'big expenditure'? answer yes or no. assistant: yes....


...

Batch starting row 472: EN = effective step -> Prediction: 1, Generated: user:  
does this emoji sequence mean 'effective step'? answer yes or no. assistant: no...


Processing Batches:  98%|█████████▊| 61/62 [8:52:56<11:53, 713.11s/it]

Batch starting row 480: EN = effective reprimand -> Prediction: 1, Generated: user:  
does this emoji sequence mean 'effective reprimand'? answer yes or no. assistant: no....


Processing Batches: 100%|██████████| 62/62 [8:59:10<00:00, 521.79s/it]

Batch starting row 488: EN = effective entrance -> Prediction: 1, Generated: user:  
does this emoji sequence mean 'effective entrance'? answer yes or no. assistant: no....
Finished prediction.
Accuracy: 0.4949290060851927
F1 Macro: 0.3310719131614654


save predictions

In [11]:
generated_texts

["user:  \ndoes this emoji sequence mean 'big business'? answer yes or no. assistant: yes.",
 "user:  \ndoes this emoji sequence mean 'big business'? answer yes or no. assistant: yes.",
 "user:  \ndoes this emoji sequence mean 'big business'? answer yes or no. assistant: no.",
 "user:  \ndoes this emoji sequence mean 'big business'? answer yes or no. assistant: no.",
 "user:  \ndoes this emoji sequence mean 'big business'? answer yes or no. assistant: no.",
 "user:  \ndoes this emoji sequence mean 'big business'? answer yes or no. assistant: yes.",
 "user:  \ndoes this emoji sequence mean 'big business'? answer yes or no. assistant: no.",
 "user:  \ndoes this emoji sequence mean 'big business'? answer yes or no. assistant: no.",
 "user:  \ndoes this emoji sequence mean 'big expenditure'? answer yes or no. assistant: yes.",
 "user:  \ndoes this emoji sequence mean 'big expenditure'? answer yes or no. assistant: yes.",
 "user:  \ndoes this emoji sequence mean 'big expenditure'? answer ye

In [15]:
cleaned_generated_texts = [text.replace('\n', '') for text in generated_texts]
cleaned_generated_texts

["user:  does this emoji sequence mean 'big business'? answer yes or no. assistant: yes.",
 "user:  does this emoji sequence mean 'big business'? answer yes or no. assistant: yes.",
 "user:  does this emoji sequence mean 'big business'? answer yes or no. assistant: no.",
 "user:  does this emoji sequence mean 'big business'? answer yes or no. assistant: no.",
 "user:  does this emoji sequence mean 'big business'? answer yes or no. assistant: no.",
 "user:  does this emoji sequence mean 'big business'? answer yes or no. assistant: yes.",
 "user:  does this emoji sequence mean 'big business'? answer yes or no. assistant: no.",
 "user:  does this emoji sequence mean 'big business'? answer yes or no. assistant: no.",
 "user:  does this emoji sequence mean 'big expenditure'? answer yes or no. assistant: yes.",
 "user:  does this emoji sequence mean 'big expenditure'? answer yes or no. assistant: yes.",
 "user:  does this emoji sequence mean 'big expenditure'? answer yes or no. assistant: ye

In [ ]:
test_dataset['predicted_label'] = predictions
test_dataset['generated_text'] = cleaned_generated_texts
test_dataset.to_csv("results.csv", index=False)

In [32]:
import pandas as pd

cleaned_generated_texts = pd.read_csv("results.csv")
cleaned_generated_texts["generated_text"][0]

"user:  does this emoji sequence mean 'big business'? answer yes or no. assistant: yes."

In [33]:
predictions2 = []
for text in cleaned_generated_texts["generated_text"]:
    parts = text.split("assistant:")
    assistant_response = parts[-1].strip() if len(parts) > 1 else text.strip()
    prediction = 1 if "yes" in assistant_response.lower() else 0

    print(parts, prediction)
    predictions2.append(prediction)

["user:  does this emoji sequence mean 'big business'? answer yes or no. ", ' yes.'] 1
["user:  does this emoji sequence mean 'big business'? answer yes or no. ", ' yes.'] 1
["user:  does this emoji sequence mean 'big business'? answer yes or no. ", ' no.'] 0
["user:  does this emoji sequence mean 'big business'? answer yes or no. ", ' no.'] 0
["user:  does this emoji sequence mean 'big business'? answer yes or no. ", ' no.'] 0
["user:  does this emoji sequence mean 'big business'? answer yes or no. ", ' yes.'] 1
["user:  does this emoji sequence mean 'big business'? answer yes or no. ", ' no.'] 0
["user:  does this emoji sequence mean 'big business'? answer yes or no. ", ' no.'] 0
["user:  does this emoji sequence mean 'big expenditure'? answer yes or no. ", ' yes.'] 1
["user:  does this emoji sequence mean 'big expenditure'? answer yes or no. ", ' yes.'] 1
["user:  does this emoji sequence mean 'big expenditure'? answer yes or no. ", ' yes.'] 1
["user:  does this emoji sequence mean 

In [34]:
from sklearn.metrics import accuracy_score, f1_score
true_labels = cleaned_generated_texts["label"]
print(f"Accuracy: {accuracy_score(true_labels, predictions2)}")
print(f"F1 Macro: {f1_score(true_labels, predictions2, average='macro')}")

Accuracy: 0.6734279918864098
F1 Macro: 0.6447824783283882


In [35]:
test_dataset = pd.read_csv("generated_img_dataset/test.csv")


In [36]:
test_dataset['predicted_label'] = predictions2
test_dataset['generated_text'] = cleaned_generated_texts["generated_text"]
test_dataset.to_csv("results2.csv", index=False)